In [2]:
import pandas as pd

In [3]:
df = pd.read_pickle('data/processed.pickle')
df.head()

,key,title,authors,abstract,year_published,tags,category,skills,skill_cat,types,lang_family
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,2021,"[approach:fully_automated, data_available:Fals...",[dynamic],[correctness],[correctness_dynamic],"[description, evaluation]",[OOP]
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,2021,"[approach:semi_automatic, interaction:unknown,...","[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]","[description, evaluation]",[OOP]
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,2021,"[skill:readability, approach:fully_automated, ...",[static],"[readability, maintainability]","[maintainability_static, readability_static]",[description],[OOP]
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,2021,"[approach:semi_automatic, data_available:False...","[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]","[description, evaluation]",[Unknown]
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,2021,"[approach:fully_automated, data_available:Fals...","[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]","[description, evaluation]",[OOP]


# Discussion Stats
## Grading
### Correctness

In [4]:
correct_df = df.copy(deep=True)
correct_df.skills = correct_df.skills.astype(str)
correct_df = correct_df[correct_df.skills.str.contains("^\['correctness'\]$")]
correct_df['tech'] = correct_df.tags.apply(lambda labels: [label for label in labels if label.startswith('technique')])
correct_df[['key', 'title', 'skill_cat', 'tech']]

,key,title,skill_cat,tech
0,rayyan-354359269,An experience of automated assessment in a lar...,[correctness_dynamic],"[technique:unit_testing, technique:pre_defined..."
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"[correctness_dynamic, correctness_static]","[technique:model_solution_req, technique:patte..."
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"[correctness_dynamic, correctness_static]","[technique:output_matching, technique:pattern_..."
4,rayyan-354359273,Analysis of an automatic grading system within...,"[correctness_dynamic, correctness_static]","[technique:pattern_matching, technique:unit_te..."
5,rayyan-354359274,Evaluating Control-Flow Graph Similarity for G...,[correctness_static],"[technique:model_solution_req, technique:patte..."
...,...,...,...,...
114,rayyan-354359408,ArTEMiS - An automatic assessment management s...,"[correctness_dynamic, correctness_static]","[technique:ci_cd, technique:unit_testing, tech..."
115,rayyan-354359409,Linkage objects for generalized instruction in...,"[correctness_ml, correctness_static]","[technique:machine_learning, technique:pattern..."
116,rayyan-354359410,Automated data-driven hints for computer progr...,"[correctness_dynamic, correctness_static]","[technique:unit_testing, technique:static_anal..."
117,rayyan-354359411,Learning and teaching numerical methods with a...,"[correctness_dynamic, correctness_static]","[technique:matching_rules, technique:unit_test..."


In [5]:
(correct_df.shape[0] / df.shape[0]) * 100

66.11570247933885

In [6]:
correct_df.skill_cat.value_counts()

[correctness_dynamic]                        36
[correctness_dynamic, correctness_static]    21
[correctness_static]                         17
[correctness_ml, correctness_static]          3
[correctness_ml]                              2
[correctness_dynamic, correctness_ml]         1
Name: skill_cat, dtype: int64

In [7]:
(correct_df.category.value_counts() / correct_df.shape[0]) * 100

[dynamic]            45.00
[dynamic, static]    26.25
[static]             21.25
[static, ml]          3.75
[ml]                  2.50
[dynamic, ml]         1.25
Name: category, dtype: float64

In [8]:
correct_df['only_dynamic'] = df.category.apply(lambda x: 'dynamic' in str(x) and 'static' not in str(x) and 'ml' not in str(x))
(correct_df[correct_df.only_dynamic].explode('tech').tech.value_counts() / correct_df[correct_df.only_dynamic].shape[0]) * 100

technique:unit_testing                100.000000
technique:ci_cd                        11.111111
technique:model_solution_req           11.111111
technique:trace_examination             5.555556
technique:property_based_testing        5.555556
technique:output_matching               5.555556
technique:pre_defined_questions         2.777778
technique:unit_test_expansion           2.777778
technique:mutation_testing              2.777778
technique:novel                         2.777778
technique:test_case_generation          2.777778
technique:gui_testing                   2.777778
technique:program_synthesis             2.777778
technique:code_repair_for_feedback      2.777778
Name: tech, dtype: float64

In [9]:
correct_df['only_static'] = df.category.apply(lambda x: 'dynamic' not in str(x) and 'static' in str(x) and 'ml' not in str(x))
(correct_df[correct_df.only_static].explode('tech').tech.value_counts() / correct_df[correct_df.only_static].shape[0]) * 100

technique:pattern_matching            70.588235
technique:model_solution_req          41.176471
technique:model_solution_closeness    35.294118
technique:program_repair              35.294118
technique:code_repair_for_feedback    29.411765
technique:clustering                  17.647059
technique:matching_rules              17.647059
technique:trace_examination           17.647059
technique:output_matching             11.764706
technique:graph_analysis              11.764706
technique:control_flow_graph           5.882353
technique:data_driven                  5.882353
technique:token_pattern_matching       5.882353
technique:knowledge_base               5.882353
technique:dynamic_analysis             5.882353
Name: tech, dtype: float64

In [10]:
correct_df['dynamic_static'] = df.category.apply(lambda x: 'dynamic' in str(x) and 'static' in str(x) and 'ml' not in str(x))
(correct_df[correct_df.dynamic_static].explode('tech').tech.value_counts() / correct_df[correct_df.dynamic_static].shape[0]) * 100

technique:unit_testing                100.000000
technique:matching_rules               33.333333
technique:pattern_matching             28.571429
technique:static_analysis              23.809524
technique:model_solution_req           19.047619
technique:style_check                  19.047619
technique:program_repair               14.285714
technique:model_solution_closeness      9.523810
technique:dynamic_analysis              9.523810
technique:output_matching               9.523810
technique:ci_cd                         9.523810
technique:fault_localisation            4.761905
technique:program_verification          4.761905
technique:code_repair_for_feedback      4.761905
technique:language_stuctures            4.761905
technique:test_case_generation          4.761905
technique:knowledge_base                4.761905
Name: tech, dtype: float64

In [11]:
correct_df['ml'] = df.category.apply(lambda x: 'ml' in str(x))
ml_df = correct_df.loc[correct_df.ml]
ml_df[['key', 'title', 'category']]

,key,title,category
13,rayyan-354359285,An Intelligent Online Judge System for Program...,"[dynamic, ml]"
19,rayyan-354359295,Convolutional neural network applied to code a...,[ml]
34,rayyan-354359310,Automatic assessment of programming assignment...,[ml]
61,rayyan-354359341,Source-Code Similarity Measurement: Syntax Tre...,"[static, ml]"
83,rayyan-354359367,Targeted Example Generation for Compilation Er...,"[static, ml]"
115,rayyan-354359409,Linkage objects for generalized instruction in...,"[static, ml]"


### Maintainability

In [12]:
maint_df = df.copy(deep=True)
maint_df.skills = maint_df.skills.astype(str)
maint_df = maint_df[maint_df.skills.str.contains("^\['maintainability'\]$")].copy()
maint_df

,key,title,authors,abstract,year_published,tags,category,skills,skill_cat,types,lang_family
92,rayyan-354359378,A Comparison of Inquiry-Based Conceptual Feedb...,"Cordova L., Carver J., Gershmel N., Walia G.",The feedback provided by current testing educa...,2021,"[skill:test_suite_quality, approach:fully_auto...",[static],['maintainability'],[maintainability_static],[evaluation],[OOP]


In [13]:
maint_df['tech'] = maint_df.tags.apply(lambda labels: [label for label in labels if label.startswith('technique')])
(maint_df.explode('tech').tech.value_counts() / maint_df.shape[0]) * 100

technique:model_solution_req    100.0
technique:static_analysis       100.0
Name: tech, dtype: float64

### Readability

In [14]:
# Analysing hint based problem solving strategy among novice programmers through gamification technique - Might be removed
read_df = df.copy(deep=True)
read_df.skills = read_df.skills.astype(str)
read_df = read_df[read_df.skills.str.contains("^\['readability'\]$")]
read_df[['key', 'title', 'category']]

,key,title,category
30,rayyan-354359306,The impact of automated code quality feedback ...,[static]
77,rayyan-354359360,Promoting Code Quality via Automated Feedback ...,[static]
79,rayyan-354359362,Measure Students’ Contribution in Web Programm...,[static]
107,rayyan-354359398,Static analyses in python programming courses,[static]


In [15]:
(read_df.shape[0] / df.shape[0]) * 100

3.3057851239669422

In [16]:
read_df.category.value_counts()

[static]    4
Name: category, dtype: int64

### Correctness, Documentation

In [17]:
doc_df = df.copy(deep=True)
doc_df.skills = doc_df.skills.astype(str)
doc_df[doc_df.skills.str.contains("^\['correctness', 'documentation'\]$")]

,key,title,authors,abstract,year_published,tags,category,skills,skill_cat,types,lang_family
29,rayyan-354359305,Developing applications to automatically grade...,Gerdes J.,There are many unique challenges associated wi...,2017,"[approach:fully_automated, data_available:Fals...",[static],"['correctness', 'documentation']","[correctness_static, documentation_static]","[description, evaluation]",[OOP]


### Combination Graders

In [18]:
(df.skills.value_counts() / df.shape[0]) * 100

[correctness]                                  66.115702
[correctness, readability]                     15.702479
[correctness, readability, maintainability]     5.785124
[correctness, maintainability]                  4.132231
[readability, maintainability]                  3.305785
[readability]                                   3.305785
[correctness, documentation]                    0.826446
[maintainability]                               0.826446
Name: skills, dtype: float64

In [19]:
(df.skill_cat.value_counts() / df.shape[0]) * 100

[correctness_dynamic]                                                 33.057851
[correctness_dynamic, correctness_static]                             17.355372
[correctness_static]                                                  14.049587
[correctness_dynamic, readability_static]                             12.396694
[maintainability_static, readability_static]                           3.305785
[readability_static]                                                   3.305785
[correctness_dynamic, maintainability_static]                          2.479339
[correctness_ml, correctness_static]                                   2.479339
[correctness_static, maintainability_static]                           1.652893
[correctness_static, readability_static]                               1.652893
[correctness_ml]                                                       1.652893
[correctness_static, documentation_static]                             0.826446
[correctness_ml, readability_static]    

In [20]:
combo_df = df.copy(deep=True)
combo_df.skills = combo_df.skills.astype(str)
combo_df[combo_df.skills.str.contains("^\['correctness', 'readability', 'maintainability']$")]

,key,title,authors,abstract,year_published,tags,category,skills,skill_cat,types,lang_family
50,rayyan-354359328,Question Independent Automated Code Analysis a...,K. K. Rai and B. Gupta and P. Shokeen and P. C...,Our first-hand experience with the recruiting ...,2019,"[skill:maintainability, type:evaluation, evalu...","[static, ml]","['correctness', 'readability', 'maintainability']","[correctness_ml, readability_static]","[description, evaluation]",[OOP]
73,rayyan-354359354,Teaching the Art of Functional Programming Usi...,"Hameer A,Pientka B",Online programming platforms have immense pote...,2019,"[technique:unit_testing, data_available:False,...","[dynamic, static]","['correctness', 'readability', 'maintainability']","[correctness_dynamic, maintainability_dynamic,...",[experience],[Functional]
95,rayyan-354359383,Automated Assessment of Learning Objectives in...,"Rump A., Fehnker A., Mader A.",Individual feedback is a core ingredient of a ...,2021,"[approach:semi_automatic, skill:readability, i...",[static],"['correctness', 'readability', 'maintainability']","[correctness_static, maintainability_static, r...","[description, evaluation]",[OOP]
98,rayyan-354359387,Student Refactoring Behaviour in a Programming...,"Keuning H., Heeren B., Jeuring J.",Producing high-quality code is essential for p...,2020,"[type:evaluation, technique:unit_testing, lang...","[dynamic, static]","['correctness', 'readability', 'maintainability']","[correctness_dynamic, readability_static]","[description, evaluation]",[OOP]
101,rayyan-354359391,The Impact of Iterative Assessment System on P...,"Yan Y.-X., Wu J.-P., Nguyen B.-A., Chen H.-M.",Automated programming assessment systems (APAS...,2020,"[skill:readability, approach:fully_automated, ...","[dynamic, static]","['correctness', 'readability', 'maintainability']","[correctness_dynamic, readability_static]",[evaluation],[OOP]
103,rayyan-354359394,RefacTutor: An Interactive Tutoring System for...,"Haendler T., Neumann G., Smirnov F.",While software refactoring is considered impor...,2019,"[skill:refactoring, approach:fully_automated, ...","[dynamic, static]","['correctness', 'readability', 'maintainability']","[correctness_dynamic, readability_static]",[description],[OOP]
109,rayyan-354359403,Teaching how to program using automated assess...,"Almeida J.B., MacEdo N., Proenca J.",Our department has long been an advocate of th...,2018,"[skill:maintainability, technique:unit_testing...","[dynamic, static]","['correctness', 'readability', 'maintainability']","[correctness_dynamic, maintainability_static]",[experience],[Functional]


### Code Repair papers

In [21]:
cr_df = df.copy()
cr_df['tech'] = cr_df.tags.apply(lambda labels: [label for label in labels if 'code_repair' in label or 'program_repair' in label])
cr_df.tech = cr_df.tech.map(lambda labels: None if len(labels) == 0 else labels)
cr_df[~cr_df.tech.isna()]

,key,title,authors,abstract,year_published,tags,category,skills,skill_cat,types,lang_family,tech
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,2021,"[approach:semi_automatic, interaction:unknown,...","[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]","[description, evaluation]",[OOP],[technique:code_repair_for_feedback]
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,2021,"[skill:readability, approach:fully_automated, ...",[static],"[readability, maintainability]","[maintainability_static, readability_static]",[description],[OOP],[technique:program_repair]
9,rayyan-354359279,Assisted learning of C programming through aut...,"Arifi S.M., Abbou R.B., Zahi A.",Programming courses are among all the current ...,2020,"[type:evaluation, technique:unit_testing, feed...","[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]","[description, evaluation]",[OOP],[technique:program_repair]
15,rayyan-354359290,Fast test suite-driven model-based fault local...,"Birch G., Fischer B., Poppleton M.","Fault localisation, i.e. the identification of...",2017,"[technique:fault_localisation, data_available:...","[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]","[description, evaluation]",[OOP],[technique:program_repair]
23,rayyan-354359299,"Search, align, and repair: Data-driven feedbac...","Wang K., Singh R., Su Z.","This paper introduces the ""Search, Align, and ...",2018,"[approach:fully_automated, interaction:unknown...",[static],[correctness],[correctness_static],"[description, evaluation]",[OOP],"[feedback:program_repair, technique:code_repai..."
27,rayyan-354359303,TipsC: Tips and corrections for programming MOOCs,"Sharma S., Agarwal P., Mor P., Karkare A.","With MOOC sizes increasing every day, improvin...",2018,"[tool:tipsc, approach:fully_automated, data_av...",[static],[correctness],[correctness_static],"[description, evaluation]",[OOP],[technique:code_repair_for_feedback]
28,rayyan-354359304,Exploring the design space of automatically sy...,"Suzuki R., Soares G., Glassman E., Head A., D'...","For massive programming classrooms, recent adv...",2017,"[data_available:False, interaction:unknown, la...",[static],[correctness],[correctness_static],[evaluation],[Unknown],"[feedback:program_repair, technique:program_re..."
51,rayyan-354359330,EvalSeer: An Intelligent Gamified System for P...,R. Nabil and N. E. Mohamed and A. Mahdy and K....,Continuous evaluation of computer programs and...,2021,"[approach:fully_automated, data_available:Fals...","[dynamic, static, ml]","[correctness, readability]","[correctness_dynamic, readability_static]",[description],[OOP],[technique:program_repair]
55,rayyan-354359334,Automatic Grading and Feedback Using Program R...,"Parihar S,Dadachanji Z,Singh PK,Das R,Karkare ...","We present GradeIT, a system that combines the...",2017,"[approach:fully_automated, data_available:Fals...","[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]","[description, evaluation]",[OOP],"[feedback:program_repair, technique:program_re..."
65,rayyan-354359346,Automated Clustering and Program Repair for In...,"Gulwani S,Radiček I,Zuleger F",Providing feedback on programming assignments ...,2018,"[approach:fully_automated, data_available:Fals...",[static],[correctness],[correctness_static],"[description, evaluation]",[OOP],"[feedback:program_repair, technique:program_re..."


### Data for supplementary material

In [34]:
df[['key', 'skill_cat']].explode('skill_cat').sort_values('skill_cat')

,key,skill_cat
0,rayyan-354359269,correctness_dynamic
81,rayyan-354359364,correctness_dynamic
40,rayyan-354359317,correctness_dynamic
39,rayyan-354359316,correctness_dynamic
38,rayyan-354359315,correctness_dynamic
...,...,...
14,rayyan-354359287,readability_static
103,rayyan-354359394,readability_static
70,rayyan-354359351,readability_static
71,rayyan-354359352,readability_static


## Techniques
### Data for supplementary material

In [26]:
def add_cats(tech):
    if tech in ['unit_testing', 'property_based_testing', 'ci', 'output_matching']:
        return 'dynamic'
    if tech in ['pattern_matching', 'static_analysis', 'code_metrics', 'cluster',
                'style_check', 'program_repair', 'rule_based', 'dsl_rules', 'model_solution_req',
                'model_solution_closeness', 'code_repair_for_feedback']:
        return 'static'
    else:
        return 'other'

In [30]:
tech_df = df.copy()
tech_df.tags = tech_df.tags.apply(lambda labels: [str(label) for label in labels if label.startswith('technique')])
tech_df = tech_df.explode('tags')
tech_df.tags = tech_df.tags.str.replace('technique:', '')
tech_df['category'] = tech_df.tags.apply(lambda tech: add_cats(tech))
tech_df[["key", 'category', "tags"]].sort_values(by=['category', 'tags'])

,key,category,tags
3,rayyan-354359272,dynamic,output_matching
9,rayyan-354359279,dynamic,output_matching
34,rayyan-354359310,dynamic,output_matching
45,rayyan-354359322,dynamic,output_matching
48,rayyan-354359326,dynamic,output_matching
...,...,...,...
90,rayyan-354359376,static,style_check
101,rayyan-354359391,static,style_check
105,rayyan-354359396,static,style_check
109,rayyan-354359403,static,style_check


## Degree of Automation

In [ ]:
doa_df = df.copy()
doa_df['approach'] = doa_df.tags.apply(lambda labels: [str(label) for label in labels if label.startswith('approach')])
doa_df = doa_df.explode('approach')
doa_df.approach.value_counts()

In [ ]:
(doa_df.approach.value_counts() /  doa_df.shape[0]) * 100

In [ ]:
(doa_df[doa_df.approach.str.contains('fully')].skills.value_counts() / doa_df.shape[0]) * 100

#### CI/CD

In [ ]:
df.explode('tags')[df.explode('tags').tags.str.contains('ci_cd')]

#### Semi

In [ ]:
doa_df[doa_df.approach.str.contains('semi')].explode('tags').tags.value_counts()

### Data for supplementary

In [ ]:
doa_df[["key", "approach"]].sort_values(by='approach')

## Language Paradigms

In [ ]:
lp_df = df.copy()
lp_df.lang_family.value_counts()

In [ ]:
(lp_df.lang_family.value_counts() / lp_df.shape[0]) * 100

In [ ]:
lp_df_query = lp_df.copy()
lp_df_query.lang_family = lp_df_query.lang_family.astype(str)
lp_df_query[lp_df_query.lang_family.str.contains('Query')]

### Data for supplementary

In [ ]:
lp_df.explode('lang_family')[["key", "lang_family"]].sort_values(by='lang_family')

## Evaluation

In [ ]:
eval_df = df.copy()
eval_df.tags = eval_df.tags.apply(lambda labels: [str(label) for label in labels if label.startswith('evaluation')])
eval_df = eval_df.explode('tags')
(eval_df.tags.value_counts() / eval_df.shape[0]) * 100

In [ ]:
eval_df[eval_df.tags.str.contains('manual_grading')]

### Data for supplementary

In [ ]:
eval_df[["key", "tags"]].sort_values(by='tags')

## Data Availability
### Data for supplementary

In [ ]:
data_df = df.copy()
data_df.tags = data_df.tags.apply(lambda labels: [str(label) for label in labels if label.startswith('data_available')])
data_df[["key", "tags"]].sort_values(by='tags')